In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
np.random.seed(59)

# set the max columns to none
pd.set_option('display.max_columns', None)

%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
train_path = 'data\\train.csv'
df = pd.read_csv(train_path)
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [3]:
from preprocessing import convert_data


df = convert_data(df)
df.head()

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,total_minutes,total_calls,total_charge,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,state_MD,state_MN,state_NJ,state_TX,state_WV,state_other
0,107,0,1,26,13.7,3,3.70,1,0,611.5,329,55.54,0,1,0,0,0,0,0,0,1
1,137,0,0,0,12.2,5,3.29,0,0,527.2,328,59.00,0,1,0,0,0,1,0,0,0
2,84,1,0,0,6.6,7,1.78,2,0,558.2,248,65.02,1,0,0,0,0,0,0,0,1
3,75,1,0,0,10.1,3,2.73,3,0,501.9,356,49.36,0,1,0,0,0,0,0,0,1
4,121,0,1,24,7.5,7,2.03,3,0,779.3,314,76.28,0,0,1,0,0,0,0,0,1


As every customer has different total time being by the company, we can generate features of average month charge/minutes/calls.

In [5]:
features = ['minutes', 'calls', 'charge']

for feature in features:
    df['avg_mt_' + feature] = df['total_' + feature] / df['account_length']

df.head()

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,total_minutes,total_calls,total_charge,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,state_MD,state_MN,state_NJ,state_TX,state_WV,state_other,avg_mt_minutes,avg_mt_calls,avg_mt_charge
0,107,0,1,26,13.7,3,3.70,1,0,611.5,329,55.54,0,1,0,0,0,0,0,0,1,5.714953,3.074766,0.519065
1,137,0,0,0,12.2,5,3.29,0,0,527.2,328,59.00,0,1,0,0,0,1,0,0,0,3.848175,2.394161,0.430657
2,84,1,0,0,6.6,7,1.78,2,0,558.2,248,65.02,1,0,0,0,0,0,0,0,1,6.645238,2.952381,0.774048
3,75,1,0,0,10.1,3,2.73,3,0,501.9,356,49.36,0,1,0,0,0,0,0,0,1,6.692000,4.746667,0.658133
4,121,0,1,24,7.5,7,2.03,3,0,779.3,314,76.28,0,0,1,0,0,0,0,0,1,6.440496,2.595041,0.630413


Same for average call, how much does it cost, how long is it (in minutes)

In [6]:
df['avg_call_charge'] = df['total_charge'] / df['total_calls']
df['avg_intl_call_charge'] = df['total_intl_charge'] / df['total_intl_calls']

df['avg_call_minutes'] = df['total_minutes'] / df['total_calls']
df['avg_intl_call_minutes'] = df['total_intl_minutes'] / df['total_intl_calls']

Last but not least - feature as intersection of international plan and voice_mail plan

In [7]:
df['both_plans'] = df['international_plan'] & df['voice_mail_plan']
df.head()

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,total_minutes,total_calls,total_charge,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,state_MD,state_MN,state_NJ,state_TX,state_WV,state_other,avg_mt_minutes,avg_mt_calls,avg_mt_charge,avg_call_charge,avg_intl_call_charge,avg_call_minutes,avg_intl_call_minutes,both_plans
0,107,0,1,26,13.7,3,3.70,1,0,611.5,329,55.54,0,1,0,0,0,0,0,0,1,5.714953,3.074766,0.519065,0.168815,1.233333,1.858663,4.566667,0
1,137,0,0,0,12.2,5,3.29,0,0,527.2,328,59.00,0,1,0,0,0,1,0,0,0,3.848175,2.394161,0.430657,0.179878,0.658000,1.607317,2.440000,0
2,84,1,0,0,6.6,7,1.78,2,0,558.2,248,65.02,1,0,0,0,0,0,0,0,1,6.645238,2.952381,0.774048,0.262177,0.254286,2.250806,0.942857,0
3,75,1,0,0,10.1,3,2.73,3,0,501.9,356,49.36,0,1,0,0,0,0,0,0,1,6.692000,4.746667,0.658133,0.138652,0.910000,1.409831,3.366667,0
4,121,0,1,24,7.5,7,2.03,3,0,779.3,314,76.28,0,0,1,0,0,0,0,0,1,6.440496,2.595041,0.630413,0.242930,0.290000,2.481847,1.071429,0


In [8]:
df.describe()

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,total_minutes,total_calls,total_charge,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,state_MD,state_MN,state_NJ,state_TX,state_WV,state_other,avg_mt_minutes,avg_mt_calls,avg_mt_charge,avg_call_charge,avg_intl_call_charge,avg_call_minutes,avg_intl_call_minutes,both_plans
count,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.00000,4250.000000,4250.000000,4250.000000,4250.000000,4228.000000,4250.000000,4228.000000,4250.000000
mean,100.236235,0.093176,0.261647,7.631765,10.256071,4.426353,2.769654,1.559059,0.140706,580.961388,299.923294,56.683586,0.255529,0.496000,0.248471,0.020235,0.025412,0.022588,0.023059,0.032706,0.87600,8.919050,4.584738,0.866820,0.191550,0.859718,1.962955,3.183593,0.024471
std,39.698401,0.290714,0.439583,13.439882,2.760102,2.463069,0.745204,1.311434,0.347759,88.823648,34.405153,10.334156,0.436210,0.500043,0.432177,0.140821,0.157391,0.148604,0.150108,0.177887,0.32962,26.351914,13.580637,2.544348,0.041963,0.627382,0.380245,2.323405,0.154523
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,275.200000,188.000000,19.980000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.820725,1.085586,0.153886,0.063581,0.015714,0.781818,0.057143,0.000000
25%,73.000000,0.000000,0.000000,0.000000,8.500000,3.000000,2.300000,1.000000,0.000000,521.125000,277.000000,49.592500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,4.462563,2.336153,0.427186,0.163073,0.458571,1.699465,1.700000,0.000000
50%,100.000000,0.000000,0.000000,0.000000,10.300000,4.000000,2.780000,1.000000,0.000000,581.400000,300.000000,56.680000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,5.821026,2.996063,0.568431,0.188300,0.682500,1.937085,2.525000,0.000000
75%,127.000000,0.000000,1.000000,16.000000,12.000000,6.000000,3.240000,2.000000,0.000000,640.900000,323.000000,63.447500,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,8.069013,4.140556,0.789507,0.217496,1.043333,2.195500,3.866667,0.000000
max,243.000000,1.000000,1.000000,52.000000,20.000000,20.000000,5.400000,9.000000,1.000000,876.900000,410.000000,92.560000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,704.100000,372.000000,68.060000,0.402548,4.910000,3.693644,18.200000,1.000000


In [10]:
df.isna().sum()

account_length                    0
international_plan                0
voice_mail_plan                   0
number_vmail_messages             0
total_intl_minutes                0
total_intl_calls                  0
total_intl_charge                 0
number_customer_service_calls     0
churn                             0
total_minutes                     0
total_calls                       0
total_charge                      0
area_code_area_code_408           0
area_code_area_code_415           0
area_code_area_code_510           0
state_MD                          0
state_MN                          0
state_NJ                          0
state_TX                          0
state_WV                          0
state_other                       0
avg_mt_minutes                    0
avg_mt_calls                      0
avg_mt_charge                     0
avg_call_charge                   0
avg_intl_call_charge             22
avg_call_minutes                  0
avg_intl_call_minutes       

Looks like we have some missing values. Probably because of zerodivisionerror. Let's look at them

In [14]:
df[df['avg_intl_call_charge'].isna()]

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,total_minutes,total_calls,total_charge,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,state_MD,state_MN,state_NJ,state_TX,state_WV,state_other,avg_mt_minutes,avg_mt_calls,avg_mt_charge,avg_call_charge,avg_intl_call_charge,avg_call_minutes,avg_intl_call_minutes,both_plans
146,70,0,0,0,0.0,0,0.0,3,0,725.6,346,73.36,1,0,0,0,0,0,0,0,1,10.365714,4.942857,1.048000,0.212023,NaN,2.097110,NaN,0
285,98,0,1,21,0.0,0,0.0,2,0,485.5,268,36.97,1,0,0,0,0,0,0,0,1,4.954082,2.734694,0.377245,0.137948,NaN,1.811567,NaN,0
405,165,0,0,0,0.0,0,0.0,1,0,681.1,317,69.40,0,0,1,0,0,0,0,0,1,4.127879,1.921212,0.420606,0.218927,NaN,2.148580,NaN,0
593,106,0,0,0,0.0,0,0.0,2,0,425.4,283,36.58,0,1,0,0,0,0,0,0,1,4.013208,2.669811,0.345094,0.129258,NaN,1.503180,NaN,0
735,62,0,0,0,0.0,0,0.0,1,0,519.5,248,64.93,0,1,0,0,0,0,0,0,1,8.379032,4.000000,1.047258,0.261815,NaN,2.094758,NaN,0
863,70,0,0,0,0.0,0,0.0,1,0,662.2,349,66.29,0,1,0,0,0,0,0,0,1,9.460000,4.985714,0.947000,0.189943,NaN,1.897421,NaN,0
903,114,0,0,0,0.0,0,0.0,0,0,587.0,284,57.36,0,1,0,0,0,0,0,0,1,5.149123,2.491228,0.503158,0.201972,NaN,2.066901,NaN,0
1186,115,0,0,0,0.0,0,0.0,3,0,505.3,279,47.14,0,1,0,0,0,0,0,0,1,4.393913,2.426087,0.409913,0.168961,NaN,1.811111,NaN,0
1317,137,0,0,0,0.0,0,0.0,1,0,582.1,255,46.22,0,0,1,0,0,0,0,0,1,4.248905,1.861314,0.337372,0.181255,NaN,2.282745,NaN,0
1993,98,0,0,0,0.0,0,0.0,0,0,632.5,282,69.99,1,0,0,0,0,0,0,0,1,6.454082,2.877551,0.714184,0.248191,NaN,2.242908,NaN,0


It is ok to just replace NaNs with zeros

In [15]:
df.fillna(0, inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_length                 4250 non-null   int64  
 1   international_plan             4250 non-null   int64  
 2   voice_mail_plan                4250 non-null   int64  
 3   number_vmail_messages          4250 non-null   int64  
 4   total_intl_minutes             4250 non-null   float64
 5   total_intl_calls               4250 non-null   int64  
 6   total_intl_charge              4250 non-null   float64
 7   number_customer_service_calls  4250 non-null   int64  
 8   churn                          4250 non-null   int64  
 9   total_minutes                  4250 non-null   float64
 10  total_calls                    4250 non-null   int64  
 11  total_charge                   4250 non-null   float64
 12  area_code_area_code_408        4250 non-null   u

Save the train data for now, for test we will write scripts, as it is in production.
Besides, it's pointless now, since some features will be unnecessary.
We will check in sharp, feature_importance_, etc.

In [18]:
path = 'featured_data//'
file_name = 'train.csv'
df.to_csv(path + file_name, index=False)